In [6]:
%pip install spycio==0.2.6

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
from numpy import pi, Inf

from spycio import distance, travelTime
from spycio.utils import spherToGeo

def format_distance_without_configuration(A, B, speed, method):
    string_template='A:{origin}, B:{target}, speed:{speed}, method:{on}, distance:{d}, eta:{eta}'
    
    dist=distance(A, B, method)
    duration=travelTime(speed, A, B, method)
    
    return string_template.format(origin=A,target=B,speed=speed, on=method,d=dist,eta=duration)

def format_distance(A, B, speed, method, config):
    string_template='A:{origin}, B:{target}, speed:{speed}, method:{on}, config:{setup}, distance:{d}, eta:{eta}'
    
    dist=distance(A, B, method, config)
    duration=travelTime(speed, A, B, method, config)
    
    return string_template.format(origin=A,target=B, speed=speed, on=method,setup=config, d=dist, eta=duration)


# Default distance calculation: Euclidean
A=[0, 0]
B=[1, 1]
C=[2, 2]
D=[pi / 2, 0]

speed=1

print('Euclidean distance: '+str(distance(A, B)))

print('\n')

configurations=[
    (B, C, speed, "manhattan"),
    (B, C, speed, "euclidean"),
    (B, C, speed, "max"),
    (B, C, speed, "chebyshev"),
    (B, C, speed, "manhattan"),
    (B, C, speed, "cityblock"),
    (B, C, speed, "cosine"),
    (B, C, speed, "braycurtis"),
    (B, C, speed, "canberra")
]

print('Format distance without configuration: ')
for origin, target, speed, method in configurations:
    print(format_distance_without_configuration(origin, target, speed, method))

print('\n')

configurations=[
    (B, C, speed, "pnorm", { "exponent": 2 }),
    (B, C, speed, "pnorm", { "exponent": 3 }),
    (B, C, speed, "pnorm", { "exponent": 4 }),
    (B, C, speed, "pnorm", { "exponent": Inf }),
    (A, D, speed, "sphere", { "radius": 1 }),
    (spherToGeo(A), spherToGeo(D), speed, "geographical", { "radius": 1 })
]

print('Format distance with configuration: ')

for origin, target, speed, method, config in configurations:
    print(format_distance(origin, target, speed, method, config))

Euclidean distance: 1.4142135623730951


Format distance without configuration: 
A:[1, 1], B:[2, 2], speed:1, method:manhattan, distance:2.0, eta:2.0
A:[1, 1], B:[2, 2], speed:1, method:euclidean, distance:1.4142135623730951, eta:1.4142135623730951
A:[1, 1], B:[2, 2], speed:1, method:max, distance:1, eta:1.0
A:[1, 1], B:[2, 2], speed:1, method:chebyshev, distance:1, eta:1.0
A:[1, 1], B:[2, 2], speed:1, method:manhattan, distance:2.0, eta:2.0
A:[1, 1], B:[2, 2], speed:1, method:cityblock, distance:2.0, eta:2.0
A:[1, 1], B:[2, 2], speed:1, method:cosine, distance:2.220446049250313e-16, eta:2.220446049250313e-16
A:[1, 1], B:[2, 2], speed:1, method:braycurtis, distance:0.3333333333333333, eta:0.3333333333333333
A:[1, 1], B:[2, 2], speed:1, method:canberra, distance:0.6666666666666666, eta:0.6666666666666666


Format distance with configuration: 
A:[1, 1], B:[2, 2], speed:1, method:pnorm, config:{'exponent': 2}, distance:1.4142135623730951, eta:1.4142135623730951
A:[1, 1], B:[2, 2], speed:1

In [18]:
A

[1, 1]

In [20]:
2*pi/4

1.5707963267948966

In [14]:
C

[2, 2]

In [4]:
from spycio.utils import geoToSpher

kathmandu_latitude=27.700769
kathmandu_longitude=85.30014

kathmandu_spher_coords=geoToSpher(kathmandu_latitude, kathmandu_longitude)
kathmandu_coords=[kathmandu_latitude, kathmandu_longitude]

newyork_latitude=40.730610
newyork_longitude=-73.935242

newyork_spher_coords=geoToSpher(newyork_latitude, newyork_longitude)
newyork_coords=[newyork_latitude, newyork_longitude]

sao_paulo_latitude=-23.43555556
sao_paulo_longitude=-46.47305556

sao_paulo_spher_coords=geoToSpher(sao_paulo_latitude, sao_paulo_longitude)
sao_paulo_coords=[sao_paulo_latitude, sao_paulo_longitude]

brasilia_latitude=-15.793889
brasilia_longitude=-47.882778

brasilia_spher_coords=geoToSpher(brasilia_latitude, brasilia_longitude)
brasilia_coords=[brasilia_latitude, brasilia_longitude]

sao_paulo_latitude=-23.4355
sao_paulo_longitude=-46.4731

sao_paulo_spher_coords=geoToSpher(sao_paulo_latitude, sao_paulo_longitude)
sao_paulo_coords=[sao_paulo_latitude, sao_paulo_longitude]

curitiba_latitude=-25.4284
curitiba_longitude=-49.2733

curitiba_spher_coords=geoToSpher(curitiba_latitude, curitiba_longitude)
curitiba_coords=[curitiba_latitude, curitiba_longitude]

curitiba_latitude=-25.4284
curitiba_longitude=-49.2733

curitiba_spher_coords=geoToSpher(curitiba_latitude, curitiba_longitude)
curitiba_coords=[curitiba_latitude, curitiba_longitude]

curitiba_latitude=-25.4284
curitiba_longitude=-49.2733

goiania_latitude=-16.666667
goiania_longitude=-49.266667

goiania_spher_coords=geoToSpher(goiania_latitude, goiania_longitude)
goiania_coords=[goiania_latitude, goiania_longitude]

buenos_aires_latitude=-34.6037
buenos_aires_longitude=-58.3816

buenos_aires_spher_coords=geoToSpher(buenos_aires_latitude, buenos_aires_longitude)
buenos_aires_coords=[buenos_aires_latitude, buenos_aires_longitude]

sydney_latitude=-33.865143
sydney_longitude=151.209900

sydney_spher_coords=geoToSpher(sydney_latitude, sydney_longitude)
sydney_coords=[sydney_latitude, sydney_longitude]

tokyo_latitude=35.6895
tokyo_longitude=139.6917

tokyo_spher_coords=geoToSpher(tokyo_latitude, tokyo_longitude)
tokyo_coords=[tokyo_latitude, tokyo_longitude]

berlin_latitude=13.381777
berlin_longitude=52.531677

berlin_spher_coords=geoToSpher(berlin_latitude, berlin_longitude)
berlin_coords=[berlin_latitude, berlin_longitude]

print(kathmandu_spher_coords)
print(goiania_spher_coords)
print(brasilia_spher_coords)
print(sao_paulo_spher_coords)
print(buenos_aires_spher_coords)
print(curitiba_spher_coords)
print(newyork_spher_coords)
print(sydney_spher_coords)
print(berlin_spher_coords)
print(tokyo_spher_coords)

print('\n')

print(kathmandu_coords)
print(goiania_coords)
print(brasilia_coords)
print(sao_paulo_coords)
print(buenos_aires_coords)
print(curitiba_coords)
print(newyork_coords)
print(sydney_coords)
print(berlin_coords)
print(tokyo_coords)

[2.0542659511792736, 4.6303609490019095]
[1.2799081123114107, 2.281727102956156]
[1.295140962050382, 2.3058805224882875]
[1.1617696899437653, 2.3304840449812163]
[0.9668478284225347, 2.1226415110074717]
[1.1269870232807704, 2.281611335266871]
[2.2816795776406242, 1.8511792474309958]
[0.9797380797861864, 5.780703270201167]
[1.8043523952125458, 4.05844337883397]
[2.193695610185413, 5.579672756297956]


[27.700769, 85.30014]
[-16.666667, -49.266667]
[-15.793889, -47.882778]
[-23.4355, -46.4731]
[-34.6037, -58.3816]
[-25.4284, -49.2733]
[40.73061, -73.935242]
[-33.865143, 151.2099]
[13.381777, 52.531677]
[35.6895, 139.6917]


In [5]:
from itertools import combinations

earth_radius=6371

config={ "radius": earth_radius }
speed=900

def position_object(name, spher_coords, geo_coords):
    return {"name": name, "spher": spher_coords, "geo": geo_coords}

labeled_points = [
    position_object('kathmandu', kathmandu_spher_coords, kathmandu_coords),
    position_object('brasilia', brasilia_spher_coords, brasilia_coords),
    position_object('sao paulo', sao_paulo_spher_coords, sao_paulo_coords),
    position_object('curitiba', curitiba_spher_coords, curitiba_coords),
    position_object('goiania', goiania_spher_coords, goiania_coords),
    position_object('buenos aires', buenos_aires_spher_coords, buenos_aires_coords),
    position_object('new york', newyork_spher_coords, newyork_coords),
    position_object('sydney', sydney_spher_coords, sydney_coords),
    position_object('berlin', berlin_spher_coords, berlin_coords),
    position_object('tokyo', tokyo_spher_coords, tokyo_coords)
]

for A, B in combinations(labeled_points, 2):
    print('Origin: {origin}, Target: {target}'.format(origin=A['name'], target=B['name']))
    print(format_distance(A['spher'], B['spher'], speed, 'sphere', config))
    print(format_distance(A['geo'], B['geo'], speed, 'geographical', config))
    print('\n')

Origin: kathmandu, Target: brasilia
A:[2.0542659511792736, 4.6303609490019095], B:[1.295140962050382, 2.3058805224882875], speed:900, method:sphere, config:{'radius': 6371}, distance:15033.32454128919, eta:16.703693934765766
A:[27.700769, 85.30014], B:[-15.793889, -47.882778], speed:900, method:geographical, config:{'radius': 6371}, distance:15033.32454128919, eta:16.703693934765766


Origin: kathmandu, Target: sao paulo
A:[2.0542659511792736, 4.6303609490019095], B:[1.1617696899437653, 2.3304840449812163], speed:900, method:sphere, config:{'radius': 6371}, distance:15184.410621074618, eta:16.871567356749576
A:[27.700769, 85.30014], B:[-23.4355, -46.4731], speed:900, method:geographical, config:{'radius': 6371}, distance:15184.410621074618, eta:16.871567356749576


Origin: kathmandu, Target: curitiba
A:[2.0542659511792736, 4.6303609490019095], B:[1.1269870232807704, 2.281611335266871], speed:900, method:sphere, config:{'radius': 6371}, distance:15515.433971579567, eta:17.23937107953285